# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words thanks to vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which is the ÌMDB dataset: it is comprised of sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=20)

2022-11-18 12:26:39.620007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 12:26:41.139650: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-18 12:26:41.139678: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-18 12:26:41.345134: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-18 12:26:44.753740: W tensorflow/stream_executor/platform/de

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown here: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with word2vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with word2vec)
- the representation learned by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As in this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. The third line stores the words and their trained embeddings in `wv`.

In [ ]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [89]:
wv['fish']

array([-0.18555415,  0.10899357,  0.09529596,  0.21006908,  0.03266552,
       -0.1487612 ,  0.03862127,  0.40991676,  0.0096422 , -0.13902563,
       -0.00496656, -0.24978283,  0.11488634, -0.10146506,  0.03669976,
       -0.08791869,  0.1752047 , -0.24120834,  0.03771476, -0.32510963,
        0.27265993, -0.05120822,  0.15271363, -0.23537397, -0.12767695,
        0.14630228, -0.24292222, -0.00990323, -0.25528783,  0.00535264,
       -0.11564039,  0.00885742, -0.08475363, -0.57552093, -0.00970419,
        0.19945437,  0.07050998, -0.15064691, -0.1897304 , -0.26827988,
        0.01490845, -0.04842067, -0.18958963,  0.05374971,  0.217391  ,
       -0.20661396,  0.07526907, -0.23486471,  0.09187578,  0.12827766,
        0.17444518, -0.13193567, -0.11724661, -0.04523108, -0.18964343,
        0.09626861,  0.00511134,  0.1949457 , -0.24959335,  0.0886929 ,
       -0.00381385,  0.11584627, -0.26056194,  0.068836  , -0.12340288,
        0.08839175,  0.00842832,  0.1823956 , -0.14781936,  0.22

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [28]:
print('emb:',wv.vector_size,'/ size:', len(wv['dog']))

emb: 100 / size: 100


How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the [`Word2Vec.wv.most_similar`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words that have similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [32]:
wv.most_similar('fish', topn=10)

[('naked', 0.9327421188354492),
 ('bowl', 0.9320507645606995),
 ('water', 0.9318605661392212),
 ('step', 0.9306609034538269),
 ('woods', 0.9302541017532349),
 ('flying', 0.930204451084137),
 ('castle', 0.9290191531181335),
 ('chasing', 0.9281830191612244),
 ('field', 0.9279825687408447),
 ('arms', 0.9272319674491882)]

Similarly to `most_similar` used on words directly, we can use [`similar_by_vector`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [35]:
wv.similar_by_vector('fish', topn=10)

[('naked', 0.9327421188354492),
 ('bowl', 0.9320507645606995),
 ('water', 0.9318605661392212),
 ('step', 0.9306609034538269),
 ('woods', 0.9302541017532349),
 ('flying', 0.930204451084137),
 ('castle', 0.9290191531181335),
 ('chasing', 0.9281830191612244),
 ('field', 0.9279825687408447),
 ('arms', 0.9272319674491882)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [36]:
wv['good'] - wv['bad']

array([-0.1671099 ,  1.1158203 , -0.39196438, -0.6045514 ,  0.2673478 ,
       -0.3255365 ,  0.35490465, -0.14744657, -0.25390768, -0.3512666 ,
        0.18060508,  0.59908843,  0.93022525, -0.01404846, -0.43073225,
        0.562472  ,  0.8929522 ,  0.567627  , -1.7305863 , -0.90828276,
       -0.65937734, -0.99592423, -0.12355989, -1.6547644 , -0.45102215,
        0.09598911, -0.17446756,  0.89135325, -0.9527452 , -0.53256273,
       -0.7592648 ,  0.6549697 , -0.16344091, -1.0091476 ,  0.37027043,
       -1.4357855 , -0.48360187,  0.56829107,  0.5836082 ,  1.4902682 ,
       -0.22851928,  0.09452033,  0.6268291 , -0.1348561 ,  0.2329433 ,
       -0.075966  ,  0.74993813,  0.48231733,  0.39558542,  0.99268067,
        0.40950957,  0.41906416, -0.24811503, -1.3957932 ,  0.5507777 ,
       -0.49440175, -0.06030886,  0.49095526, -0.8295228 , -0.05534089,
       -0.2414721 ,  0.17141306,  0.6042563 ,  0.12794095,  0.07990056,
       -0.69703877,  0.47679806, -0.7398852 ,  0.574306  , -0.07

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [38]:
res = wv['good'] - wv['bad'] + wv['stupid']
res

array([ 0.44050246,  0.31238157, -0.16191596, -0.9063362 ,  0.5082631 ,
       -1.168242  , -0.45060086, -0.04305446, -0.6943519 , -0.32487443,
       -0.31738025, -0.64160657,  1.0738465 ,  0.910401  ,  0.25829738,
        0.09859878,  0.6797076 ,  0.22416422, -1.1273248 , -1.3656604 ,
        0.8152406 , -0.8796792 ,  1.0685093 , -1.4001296 , -0.6273372 ,
       -0.18798143,  0.25578743,  0.9787762 , -0.54694307, -0.33717692,
       -0.38922292, -0.01293343,  0.18982327, -1.1306007 , -0.18964136,
       -0.5619917 , -0.49596435,  0.998189  , -0.18820012,  0.27998734,
       -0.35845673,  0.6610518 ,  0.08943897,  0.3130681 ,  0.19536799,
        0.5126014 ,  0.42738378, -0.22863048,  0.282124  ,  0.6497131 ,
        1.0863829 , -0.25579065, -0.09112154, -1.0957761 ,  0.2557321 ,
       -0.46674806,  0.15984683,  0.5523486 , -1.0194296 ,  0.24502063,
       -0.29912794, -0.5118924 ,  1.4908135 ,  0.2714255 , -0.5976806 ,
       -0.00698835,  0.7158364 , -0.29167938,  0.293115  ,  0.08

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `similar_by_vector` function) of `res`

In [39]:
wv.similar_by_vector(res, topn=10)

[('good', 0.7624704837799072),
 ('nice', 0.7359461784362793),
 ('decent', 0.6839771866798401),
 ('great', 0.632158637046814),
 ('interesting', 0.6211627721786499),
 ('cool', 0.5980461835861206),
 ('fine', 0.5947336554527283),
 ('solid', 0.5824175477027893),
 ('cute', 0.5819118022918701),
 ('strange', 0.5804674625396729)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [43]:
man1 = wv['boy'] - wv['girl'] + wv['woman']

wv.similar_by_vector(man1, topn=10)

[('woman', 0.9026809334754944),
 ('child', 0.8861483931541443),
 ('boy', 0.8568129539489746),
 ('man', 0.8490236401557922),
 ('lady', 0.8084245324134827),
 ('baby', 0.7963420152664185),
 ('doctor', 0.7825407385826111),
 ('rich', 0.777028501033783),
 ('affair', 0.7749112844467163),
 ('victim', 0.7708556056022644)]

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's check some out. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [44]:
wordvec_2 = Word2Vec(X_train,vector_size=50)

In [48]:
wv2 = wordvec_2.wv
wv2.vector_size
len(wv2['dog'])

50

❓ **Question** ❓ Use the `Word2Vec.wv.key_to_index` attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [59]:
import numpy as np
print('wv2:',len(wv2.key_to_index),'X_train:',len(X_train))

wv2: 12683 X_train: 5000


There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [60]:
word2vec_3 = Word2Vec(X_train, min_count=20,vector_size=70)
word2vec_4 = Word2Vec(X_train, min_count=2,vector_size=70)
wv3 = word2vec_3.wv
wv4 = word2vec_4.wv

In [63]:
len(wv3.key_to_index), len(wv4.key_to_index)

(4418, 24403)

Remember that we said word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [64]:
word2vec_5 = Word2Vec(X_train, window=10)
word2vec_6 = Word2Vec(X_train, window=2)

len(word2vec_5.wv.key_to_index), len(word2vec_6.wv.key_to_index)

(12683, 12683)

In [92]:
res = [word2vec.wv.get_vector(word) for word in example]

len(res[1])

100

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [2]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    embedded_sentence = [word2vec.wv.get_vector(word) for word in sentence if word in word2vec.wv]
    return np.array(embedded_sentence)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

NameError: name 'word2vec' is not defined

In [124]:
type(X_train[0])

list

In [131]:
for i in X_train:
    embed_sentence(word2vec, i)

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [ ]:
def embedding(word2vec, sentences):
    return [embed_sentence(word2vec,i) for i in sentences]



In [ ]:
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)


In [10]:
X_train[0]

['this',
 'was',
 'an',
 'absolutely',
 'terrible',
 'movie',
 "don't",
 'be',
 'lured',
 'in',
 'by',
 'christopher',
 'walken',
 'or',
 'michael',
 'ironside',
 'both',
 'are',
 'great',
 'actors',
 'but',
 'this',
 'must',
 'simply',
 'be',
 'their',
 'worst',
 'role',
 'in',
 'history',
 'even',
 'their',
 'great',
 'acting',
 'could',
 'not',
 'redeem',
 'this',
 "movie's",
 'ridiculous',
 'storyline',
 'this',
 'movie',
 'is',
 'an',
 'early',
 'nineties',
 'us',
 'propaganda',
 'piece',
 'the',
 'most',
 'pathetic',
 'scenes',
 'were',
 'those',
 'when',
 'the',
 'columbian',
 'rebels',
 'were',
 'making',
 'their',
 'cases',
 'for',
 'revolutions',
 'maria',
 'conchita',
 'alonso',
 'appeared',
 'phony',
 'and',
 'her',
 'pseudo',
 'love',
 'affair',
 'with',
 'walken',
 'was',
 'nothing',
 'but',
 'a',
 'pathetic',
 'emotional',
 'plug',
 'in',
 'a',
 'movie',
 'that',
 'was',
 'devoid',
 'of',
 'any',
 'real',
 'meaning',
 'i',
 'am',
 'disappointed',
 'that',
 'there',
 'are

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
X_train_pad = pad_sequences(X_train,dtype='float32',padding='post', value=0.0,maxlen=100)
X_test_pad = pad_sequences(X_test,dtype='float32',padding='post',value=0.0, maxlen=100)

ValueError: could not convert string to float: 'are'

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors that can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [147]:
### YOUR CODE HERE

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)

NameError: name 'X_train_pad' is not defined